In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:

import os
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

import datetime

import glob
os.environ['TOKEN'] = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'


In [ ]:
cd gdrive/My Drive/TFM/

[Errno 2] No such file or directory: 'gdrive/My Drive/TFM/'
/content/gdrive/My Drive/TFM


#classes and methods definitions

In [ ]:
def auth():
    return os.getenv('TOKEN')

In [ ]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [ ]:
def create_url(keyword, start_date, end_date, max_results = 10):
    
    search_url = "https://api.twitter.com/2/tweets/counts/all" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'granularity':'hour',                    
                    'next_token': {}}
    return (search_url, query_params)

In [ ]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [ ]:
def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. Author ID
        count = tweet['tweet_count']

        # 2. Time created
        end = dateutil.parser.parse(tweet['end'])

        res = [count,end]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter) 

#API Retrieval

In [ ]:
#Inputs for the request
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "ETH"
start_time = "2021-02-11T16:00:00.000Z"
end_time = "2021-08-29T00:00:00.000Z"

In [ ]:
url = create_url(keyword, start_time,end_time)
json_response = connect_to_endpoint(url[0], headers, url[1])

resp = pd.DataFrame.from_dict(json_response['data'])

flag = True
while(flag):
  if 'next_token' in json_response['meta']:
    print("-------------------")
    next_token = json_response['meta']['next_token']
    print("Token: ", next_token)
    url = create_url(keyword, start_time,end_time)
    json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
    aux1 = pd.DataFrame.from_dict(json_response['data'])
    resp = aux1.append(resp)
    time.sleep(5)
  else:
    break


Endpoint Response Code: 200
-------------------
Token:  1jzu9lk96azp0b3x7888hrpw8ffue8s3q5yy627kagl9
Endpoint Response Code: 200
-------------------
Token:  1jzu9lk96azp0b3x7888hrpw8ffu8rwx2eauxz460i9p
Endpoint Response Code: 200
-------------------
Token:  1jzu9lk96azp0b3x7888hrpw8ffu3b1uas0sqy6wj0ql
Endpoint Response Code: 200
-------------------
Token:  1jzu9lk96azp0b3x7888hrpw8fftxu6nn0cpiv3i92f1
Endpoint Response Code: 200
-------------------
Token:  1jzu9lk96azp0b3x7888hrpw8fftsdbkve2nbu68rkvx
Endpoint Response Code: 200
-------------------
Token:  1jzu9lk96azp0b3x7888hrpw8fftmwg6fbmi1mqkwozh
Endpoint Response Code: 200


In [ ]:
from datetime import datetime

resp['start'] = resp.start.map(lambda t: datetime.strptime(t[:-3], "%Y-%m-%dT%H:%M:%S.%f"))
resp['end'] =  resp.end.map(lambda t: datetime.strptime(t[:-3], "%Y-%m-%dT%H:%M:%S.%f"))
resp

,end,start,tweet_count
0,2021-02-11 17:00:00,2021-02-11 16:00:00,3914
1,2021-02-11 18:00:00,2021-02-11 17:00:00,4263
2,2021-02-11 19:00:00,2021-02-11 18:00:00,3691
3,2021-02-11 20:00:00,2021-02-11 19:00:00,3126
4,2021-02-11 21:00:00,2021-02-11 20:00:00,3249
...,...,...,...
739,2021-08-28 20:00:00,2021-08-28 19:00:00,19365
740,2021-08-28 21:00:00,2021-08-28 20:00:00,12631
741,2021-08-28 22:00:00,2021-08-28 21:00:00,10831
742,2021-08-28 23:00:00,2021-08-28 22:00:00,10356


In [ ]:
# raw trade data from https://public.bitmex.com/?prefix=data/trade/ 
Dollar_bars = pd.DataFrame()
for i,file in enumerate(glob.glob("data/*.csv")):
  if i == 0:
    Dollar_bars = Dollar_bars.append(pd.read_csv(file))
    print('Percentge of files already Loaded:',round((i/len(glob.glob("data/*.csv")))*100,1), '%. There are', len(glob.glob("data/*.csv"))-i, "files left", end='')
  else:
    Dollar_bars = Dollar_bars.append(pd.read_csv(file))
    print('\r Percentge of files already Loaded:',round((i/len(glob.glob("data/*.csv")))*100,1), '%. There are', len(glob.glob("data/*.csv"))-i, "files left",end='', flush=True)

Percentge of files already Loaded: 0.0 %. There are 1 files left

In [ ]:
Dollar_bars

,timestamp,open,high,low,close,grossValue,homeNotional,foreignNotional
0,2021-02-15 00:00:01.126206,1806.45,1815.40,1804.50,1814.25,23306453240,6274.508813,1.135510e+07
1,2021-02-15 00:13:14.087540,1814.25,1815.45,1806.70,1809.65,23312646485,6286.008690,1.137603e+07
2,2021-02-15 00:44:06.384729,1809.65,1815.10,1798.25,1798.25,23326174305,6285.648281,1.137575e+07
3,2021-02-15 01:10:59.281754,1798.15,1801.15,1786.40,1791.50,23369978610,6304.021926,1.130914e+07
4,2021-02-15 01:21:33.154619,1791.50,1791.50,1774.90,1776.30,23673715015,6417.108914,1.142810e+07
...,...,...,...,...,...,...,...,...
9308,2021-08-27 21:12:12.640873,3275.25,3282.45,3269.10,3279.90,23188396070,3470.098075,1.136606e+07
9309,2021-08-27 21:38:06.914887,3279.90,3285.00,3272.00,3274.45,23170388540,3463.570218,1.135689e+07
9310,2021-08-27 22:09:59.196710,3274.45,3282.00,3256.05,3279.05,23245926585,3481.498832,1.138027e+07
9311,2021-08-27 23:04:15.387247,3279.05,3284.10,3262.65,3266.25,23187488520,3473.536886,1.137270e+07


In [ ]:
import numpy as np

In [ ]:
Dollar_bars.index = Dollar_bars['timestamp']
Dollar_bars['timestamp'] = Dollar_bars.timestamp.map(lambda t: datetime.strptime(t[:-3], "%Y-%m-%d %H:%M:%S.%f"))

In [ ]:
import numpy as np
Dollar_bars['tweet_count'] = np.nan
for index1, row1 in Dollar_bars.iterrows():
  count = 0
  for index, row in resp.iterrows():
    #lagging the signal 1h for selection bias
    if (row1['timestamp']+pd.Timedelta('1h') > row['start'] and row1['timestamp']+pd.Timedelta('1h') < row['end']):
      count = row['tweet_count']
      #Dollar_bars.set_value(index1,'tweet_count',count)
      Dollar_bars.at[index1,'tweet_count'] = count
      print('\r Timestamp',row1['timestamp'], '  is in between:',row['start'],'  and:',row['end'] ,end='',flush=False)
      #print('And the number of tweets for that period is: ',count)

 Timestamp 2021-08-27 23:53:38.218000   is in between: 2021-08-28 00:00:00   and: 2021-08-28 01:00:00

In [ ]:
Dollar_bars.to_csv('Dollar_bars_tweet_counts_1_final.csv')
!cp Dollar_bars_tweet_counts_1_final.csv "gdrive/My Drive/TFM/Dollar_bars_tweet_counts_1_final.csv"

cp: cannot create regular file 'gdrive/My Drive/TFM/Dollar_bars_tweet_counts_1_final.csv': No such file or directory


#Second API retrieval

In [ ]:
#Inputs for the request
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "Ethereum"
start_time = "2021-02-13T16:00:00.000Z"
end_time = "2021-08-29T00:00:00.000Z"

In [ ]:
url = create_url(keyword, start_time,end_time)
json_response = connect_to_endpoint(url[0], headers, url[1])

resp = pd.DataFrame.from_dict(json_response['data'])

flag = True
while(flag):
  if 'next_token' in json_response['meta']:
    print("-------------------")
    next_token = json_response['meta']['next_token']
    print("Token: ", next_token)
    url = create_url(keyword, start_time,end_time)
    json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
    aux1 = pd.DataFrame.from_dict(json_response['data'])
    resp = aux1.append(resp)
    time.sleep(5)
  else:
    break


Endpoint Response Code: 200
-------------------
Token:  1jzu9lk96azp0b3x7888hrpw8ffue8s3q5yy627kagl9
Endpoint Response Code: 200
-------------------
Token:  1jzu9lk96azp0b3x7888hrpw8ffu8rwx2eauxz460i9p
Endpoint Response Code: 200
-------------------
Token:  1jzu9lk96azp0b3x7888hrpw8ffu3b1uas0sqy6wj0ql
Endpoint Response Code: 200
-------------------
Token:  1jzu9lk96azp0b3x7888hrpw8fftxu6nn0cpiv3i92f1
Endpoint Response Code: 200
-------------------
Token:  1jzu9lk96azp0b3x7888hrpw8fftsdbkve2nbu68rkvx
Endpoint Response Code: 200
-------------------
Token:  1jzu9lk96azp0b3x7888hrpw8fftmwg6fbmi1mqkwozh
Endpoint Response Code: 200


In [ ]:
from datetime import datetime

resp['start'] = resp.start.map(lambda t: datetime.strptime(t[:-3], "%Y-%m-%dT%H:%M:%S.%f"))
resp['end'] =  resp.end.map(lambda t: datetime.strptime(t[:-3], "%Y-%m-%dT%H:%M:%S.%f"))
resp

,end,start,tweet_count
0,2021-02-13 17:00:00,2021-02-13 16:00:00,1737
1,2021-02-13 18:00:00,2021-02-13 17:00:00,1661
2,2021-02-13 19:00:00,2021-02-13 18:00:00,1654
3,2021-02-13 20:00:00,2021-02-13 19:00:00,1447
4,2021-02-13 21:00:00,2021-02-13 20:00:00,1545
...,...,...,...
739,2021-08-28 20:00:00,2021-08-28 19:00:00,2817
740,2021-08-28 21:00:00,2021-08-28 20:00:00,2952
741,2021-08-28 22:00:00,2021-08-28 21:00:00,3018
742,2021-08-28 23:00:00,2021-08-28 22:00:00,3086


In [ ]:
# raw trade data from https://public.bitmex.com/?prefix=data/trade/ 
Dollar_bars = pd.DataFrame()
for i,file in enumerate(glob.glob("data/*.csv")):
  if i == 0:
    Dollar_bars = Dollar_bars.append(pd.read_csv(file))
    print('Percentge of files already Loaded:',round((i/len(glob.glob("data/*.csv")))*100,1), '%. There are', len(glob.glob("data/*.csv"))-i, "files left", end='')
  else:
    Dollar_bars = Dollar_bars.append(pd.read_csv(file))
    print('\r Percentge of files already Loaded:',round((i/len(glob.glob("data/*.csv")))*100,1), '%. There are', len(glob.glob("data/*.csv"))-i, "files left",end='', flush=True)

Percentge of files already Loaded: 0.0 %. There are 1 files left

In [ ]:
Dollar_bars.drop(columns=['timestamp.1'], inplace=True)
Dollar_bars.index = Dollar_bars['timestamp']
Dollar_bars['timestamp'] = Dollar_bars.timestamp.map(lambda t: datetime.strptime(t[:-3], "%Y-%m-%d %H:%M:%S.%f"))
Dollar_bars

In [60]:
import numpy as np
Dollar_bars['tweet_count2'] = np.nan
for index1, row1 in Dollar_bars.iterrows():
  count = 0
  for index, row in resp.iterrows():
    if (row1['timestamp']+pd.Timedelta('1h') > row['start'] and row1['timestamp']+pd.Timedelta('1h') < row['end']):
      count = row['tweet_count']
      #Dollar_bars.set_value(index1,'tweet_count',count)
      Dollar_bars.at[index1,'tweet_count2'] = count
      print('\r Timestamp',row1['timestamp'], '  is in between:',row['start'],'  and:',row['end'] ,end='',flush=False)
      #print('And the number of tweets for that period is: ',count)

 Timestamp 2021-08-27 23:53:38.218000   is in between: 2021-08-28 00:00:00   and: 2021-08-28 01:00:00

In [63]:
Dollar_bars

,timestamp,open,high,low,close,grossValue,homeNotional,foreignNotional,tweet_count,tweet_count2
timestamp,,,,,,,,,,
2021-02-15 00:00:01.126206,2021-02-15 00:00:01.126,1806.45,1815.40,1804.50,1814.25,23306453240,6274.508813,1.135510e+07,2268.0,1133.0
2021-02-15 00:13:14.087540,2021-02-15 00:13:14.087,1814.25,1815.45,1806.70,1809.65,23312646485,6286.008690,1.137603e+07,2268.0,1133.0
2021-02-15 00:44:06.384729,2021-02-15 00:44:06.384,1809.65,1815.10,1798.25,1798.25,23326174305,6285.648281,1.137575e+07,2268.0,1133.0
2021-02-15 01:10:59.281754,2021-02-15 01:10:59.281,1798.15,1801.15,1786.40,1791.50,23369978610,6304.021926,1.130914e+07,2320.0,1212.0
2021-02-15 01:21:33.154619,2021-02-15 01:21:33.154,1791.50,1791.50,1774.90,1776.30,23673715015,6417.108914,1.142810e+07,2320.0,1212.0
...,...,...,...,...,...,...,...,...,...,...
2021-08-27 21:12:12.640873,2021-08-27 21:12:12.640,3275.25,3282.45,3269.10,3279.90,23188396070,3470.098075,1.136606e+07,9336.0,3582.0
2021-08-27 21:38:06.914887,2021-08-27 21:38:06.914,3279.90,3285.00,3272.00,3274.45,23170388540,3463.570218,1.135689e+07,9336.0,3582.0
2021-08-27 22:09:59.196710,2021-08-27 22:09:59.196,3274.45,3282.00,3256.05,3279.05,23245926585,3481.498832,1.138027e+07,8944.0,3801.0


In [62]:
Dollar_bars.to_csv('Dollar_bars_tweet_counts_2_final.csv')
!cp Dollar_bars_tweet_counts_2_final.csv "gdrive/My Drive/TFM/Dollar_bars_tweet_counts_2_final.csv"

cp: cannot create regular file 'gdrive/My Drive/TFM/Dollar_bars_tweet_counts_2_final.csv': No such file or directory


In [ ]:
!lscpu |grep 'Model name'

Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz


In [ ]:
!free -h --si | awk  '/Mem:/{print $2}'

13G
